In [ ]:
"""
Sequence to sequence

입력된 정보에서 추출된 정보들을 하나의 벡터로 만듬
encoder = 전체 입력되어지는 문장에대한 의미
encoder-> context벡터 출력 -> decoder ->기계번역
context벡터 = 실수형태의 값들이 저장되어있는 벡터 : 압축되어있는 정보
encdoer 와 decorder에서는 모두 RNN 구조를 사용(hidden_stats)

*Sequence to sequence ATTENTION모델

-모든 각각에 셀에 있는 hiddenstates가 다 넘어감
-encoder 쪽에서 decoder쪽으로 모든 스텝에 해당하는 hidden states가 넘어간다
-encoder에서 전달받은 전체 hidden states를 확인한다 -> 각각의 hidden state는 이전단어에 대한 입력단어에 대한 정보도 있고
현재 단어에 대한 입력 정보도 있는데 이러한 hidden state마다 점수를 계산한다 -> 계산된 점수에 대해 softmax를 취하고
각각의 벡터하고 연산을 취함 -> 어느 단어에 집중해야할지 나옴

트랜스포머(transformer)

어텐션(attention)과 차이점 : 순환신경망(RNN, LSTM) 구조가 빠짐
인코더를 통해서 집중해야할 단어 고름
RNN에서는 웨이트 값이 똑같이 공유되나 어텐션은 웨이트 값이 서로 다름
가장 아래의 인코더에서 워드 임베딩 작업
'the animal didn't cross the street because it was too tired'
컴퓨터는 문장에서 it이 뭔지 알 수 없으므로 self-attention을 통해 animal과 it을 연결
입력문장에 다른 단어들의 도움을 받아 타겟 단어 인코딩 함
RNN의 역활을 self_attention 역활을 대신함
입력된 벡터에서 3개의 벡터가 만들어짐 (Query vector, a Key vector, and a Value vector)
"""

In [ ]:
#sequence to sequence -> 프랑스어 번역

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 227815


In [4]:
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [5]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
56895,I know you understand.,Je sais que tu comprends.
18833,You'd better go.,Tu ferais mieux d'y aller.
14035,You're amusing.,Tu me fais marrer.
6616,"Tom, wake up.","Tom, réveille-toi."
1587,They left.,Elles sont parties.
15000,He's a gardener.,Il est jardinier.
10443,Can I help you?,Puis-je t'aider ?
45518,Why is Neptune blue?,Pourquoi Neptune est-elle bleue ?
33513,I have two tickets.,J'ai deux tickets.
47085,Everyone wants money.,Chacun veut de l'argent.


In [6]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n') #시작 심볼(\t) + x + 종료 심볼(\n)
lines.sample(10)

,src,tar
34486,I'm not a deadbeat.,\t Je ne suis pas un bon à rien. \n
45835,You should be angry.,\t Vous devriez être en colère. \n
31112,You can still win.,\t Tu peux toujours gagner. \n
27475,I'll let you know.,\t Je vous le ferai savoir. \n
45165,What's Tom drinking?,\t Qu'est-ce que boit Tom ? \n
12052,Is it that bad?,\t Est-ce si mal ? \n
15971,I'll be glad to.,\t Avec plaisir. \n
33787,I must talk to you.,\t J'ai besoin de te parler. \n
36218,There's no way out.,\t Il n'y a pas de sortie. \n
44508,Tom was lost at sea.,\t Tom a disparu en mer. \n


In [ ]:
#토큰의 단위에 따라 예측 (문장 토큰 -> 문장 예측, 단어 토큰 -> 단어 예측)

In [7]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

In [9]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€'}

In [10]:
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [11]:
tar_vocab

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽'}

In [12]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [16]:
src_vocab = sorted(list(src_vocab))
src_vocab

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€']

In [ ]:
len(src_vocab)

In [17]:
tar_vocab = sorted(list(tar_vocab))
tar_vocab

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽']

In [20]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [22]:
encoder_input = []

for line in lines.src:
    encoded_line=[]
    for c in line:
        encoded_line.append(src_to_index[c])
    encoder_input.append(encoded_line)
encoder_input

[[30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [31, 58, 10],
 [31, 58, 10],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [46, 57, 64, 23],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [29, 58, 67, 54, 2],
 [31, 54, 61, 65, 2],
 [31, 58, 53, 54, 10],
 [31, 58, 53, 54, 10],
 [33, 70, 62, 65, 2],
 [33, 70, 62, 65, 10],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [25, 54, 56, 58, 63, 10],
 [25, 54, 56, 58, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 

In [23]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [24]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [25]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [26]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [27]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [28]:
encoder_input.shape

(60000, 22, 80)

In [29]:
decoder_input.shape

(60000, 76, 104)

In [30]:
decoder_target.shape

(60000, 76, 104)

In [31]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# Seq2Seq 인코더 설계

In [32]:
encoder_inputs = Input(shape=(None, src_vocab_size))

In [34]:
encoder_lstm = LSTM(units=256, return_state=True)

In [36]:
encoder_outputs, state_h, state_c =encoder_lstm(encoder_inputs) # state_h(은닉상태), state_c(셀상태)

In [37]:
encoder_states = [state_h, state_c]

In [ ]:
# Seq2Seq 디코더 설계

In [38]:
decoder_inputs = Input(shape=(None, tar_vocab_size))

In [40]:
decoder_lstm = LSTM(units=256, return_state=True, return_sequences=True)

In [42]:
decoder_outputs, _, _ =decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더는 은닉 상태와 셀 상태 정보가 필요 없음(훈련과저에서는)

In [43]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')

In [44]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [45]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [46]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [47]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 132s 173ms/step - loss: 0.8540 - val_loss: 0.7766
Epoch 2/40
750/750 [==============================] - 124s 165ms/step - loss: 0.5749 - val_loss: 0.6754
Epoch 3/40
750/750 [==============================] - 124s 165ms/step - loss: 0.5038 - val_loss: 0.6094
Epoch 4/40
750/750 [==============================] - 124s 165ms/step - loss: 0.4575 - val_loss: 0.5539
Epoch 5/40
750/750 [==============================] - 120s 161ms/step - loss: 0.4234 - val_loss: 0.5212
Epoch 6/40
750/750 [==============================] - 125s 166ms/step - loss: 0.3979 - val_loss: 0.4985
Epoch 7/40
750/750 [==============================] - 125s 167ms/step - loss: 0.3775 - val_loss: 0.4759
Epoch 8/40
750/750 [==============================] - 126s 168ms/step - loss: 0.3607 - val_loss: 0.4550
Epoch 9/40
750/750 [==============================] - 124s 166ms/step - loss: 0.3467 - val_loss: 0.4451
Epoch 10/40
750/750 [==============================] - 122s 163m

In [ ]:
'''
테스트 과정

1. 번역 대상 입력 문장이 인코더에 들어감 -> 은닉/셀 상태 -> 디코더
2. 디코더의 입력 시그널 전달(sos, \t)
3. 디코더는 다음 문자 예측
'''

In [48]:
#테스트 과정에서의 디코더 정의

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)

decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [49]:
#테스트 과정에섯의 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [56]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [64]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq, verbose=0)

  # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

    # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [65]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Courage ! 
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut ! 
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Attention ! 
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Va te concerouir. 
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Dégagez. 
